In [1]:
import cv2,os,torch
import json
import matplotlib.pyplot as plt
import numpy as np
from load_data import load_data
from torch.utils.data import Dataset,DataLoader
import albumentations as albu
import torchvision
import torch.nn as nn
import sys
sys.path.append('/home/chenyixin/chenyixin/')
import segmentor

Backbone HRNet Pretrained weights at: ./checkpoints/hrnetv2_32_model_best_epoch96.pth, only usable for HRNetv2-32


In [2]:
def load_data(path='./data',mode='train',verbose=False):
    types_dict = {'/恶性/':'malign', '/良性/':'benign', '/交界性/':'borderline'}
    imgs,masks,type_labels,pathology_labels = [],[],[],[]
    name = []
    
    for basepath,dirnames,files in os.walk(path):
        jpg = [f for f in files if '.jpg' in f]
        if len(jpg) == 0:
            continue
        for j in jpg:
            jpg_path = os.path.join(basepath,j)
            img = cv2.imread(jpg_path)

            try:
                js = json.load(open(jpg_path.replace('.jpg','.json'),'r'))
            except:
                print(jpg_path)
                break
            if len(img.shape)==2:    
                mask = np.zeros_like(img)
            elif len(img.shape) ==3:
                mask = np.zeros_like(img[:,:,0])
            for shape in js['shapes']:
                cv2.fillPoly(mask,[np.int32(shape['points'])],color=255)
                
            for t in types_dict:
                if t in basepath:
                    break
            na = basepath.replace("/Image","").split("/")[-1]
            imgs.append(img)
            masks.append(mask)
            type_labels.append(types_dict[t])
            pathology_labels.append(0)
            name.append(na)
                
    return imgs,masks,type_labels,pathology_labels,name

In [3]:
imgs,masks,type_labels,pathology_labels,name = load_data(path='./AI基线资料/灰色/',verbose=False)
c_imgs,c_masks,c_type_labels,c_pathology_labels,c_name = load_data(path='./AI基线资料/彩色/',verbose=False)

./AI基线资料/灰色/正常组/贾贞_02.201905130557.01/Images/02.201905130557.01.0007.08530900958.jpg
./AI基线资料/灰色/正常组/张娜_02.202109080922.01/Images/02.202109080922.01.21288.0008.10143400794.jpg
./AI基线资料/灰色/正常组/亢珍珍_02.202109130749.01/Images/02.202109130749.01.21288.0004.11435100718.jpg
./AI基线资料/灰色/正常组/高原_02.201906030943.01/Images/02.201906030943.01.0005.10371900248.jpg
./AI基线资料/灰色/正常组/白林花_02.201903010634.01/Images/02.201903010634.01.0005.10315400239.jpg
./AI基线资料/灰色/正常组/王焱_02.201903070242.02/Images/02.201903070242.02.0005.08341000154.jpg
./AI基线资料/灰色/正常组/张昱洁_02.201904041218.01/Images/02.201904041218.01.0006.11170800051.jpg
./AI基线资料/灰色/正常组/常凤霞_02.201906240920.01/Images/02.201906240920.01.0006.10565100859.jpg
./AI基线资料/灰色/正常组/谈蓉蓉_02.202109050272.01/Images/02.202109050272.01.21288.0001.11014700429.jpg
./AI基线资料/灰色/正常组/闫欣_02.201905091597.01/Images/02.201905091597.01.0004.15175400164.jpg
./AI基线资料/灰色/正常组/沈晓辉_02.201905211146.01/Images/02.201905211146.01.0004.10561400885.jpg
./AI基线资料/灰色/正常组/陆颖_02.201904080998.01/Ima

./AI基线资料/灰色/正常组/孙怡婕_02.201905110536.01/Images/02.201905110536.01.0004.11171900902.jpg
./AI基线资料/灰色/正常组/孟江云_02.202109090691.01/Images/02.202109090691.01.21288.0003.09142400555.jpg
./AI基线资料/灰色/正常组/代伟_02.201904150411.01/Images/02.201904150411.01.0003.08441400391.jpg
./AI基线资料/灰色/正常组/苗淑娟_02.202109010470.02/Images/02.202109010470.02.21288.0006.09023300423.jpg
./AI基线资料/灰色/正常组/陈南凤_02.201904181341.01/Images/02.201904181341.01.0008.11555300089.jpg
./AI基线资料/灰色/正常组/张婕_02.201905300555.01/Images/02.201905300555.01.0004.08484400932.jpg
./AI基线资料/灰色/正常组/李海兰_02.201905210544.01/Images/02.201905210544.01.0003.08502700751.jpg
./AI基线资料/灰色/正常组/任荣_02.201905231381.02/Images/02.201905231381.02.0005.14060600749.jpg
./AI基线资料/灰色/正常组/柳娜_02.201906272036.01/Images/02.201906272036.01.0007.16515100436.jpg
./AI基线资料/灰色/正常组/常静萍_02.202109210272.01/Images/02.202109210272.01.20210.0014.10200900756.jpg
./AI基线资料/灰色/正常组/周璞芬_02.201904260729.01/Images/02.201904260729.01.0003.10061900010.jpg
./AI基线资料/灰色/正常组/田瑞端_02.202109151909.01/I

./AI基线资料/灰色/正常组/段林志_02.202109171515.02/Images/02.202109171515.02.21288.0003.15463100881.jpg
./AI基线资料/灰色/正常组/缪莹_02.201905241478.01/Images/02.201905241478.01.0008.14521900599.jpg
./AI基线资料/灰色/正常组/何湘_02.201906200628.01/Images/02.201906200628.01.0003.08470300535.jpg
./AI基线资料/灰色/正常组/郑涛_02.201906170963.01/Images/02.201906170963.01.0004.10440200597.jpg
./AI基线资料/灰色/正常组/焦利军_02.202109090657.01/Images/02.202109090657.01.21288.0004.10211400794.jpg
./AI基线资料/灰色/正常组/王素军_02.201905231783.01/Images/02.201905231783.01.0007.15551300030.jpg
./AI基线资料/灰色/正常组/戚娟_02.201904011170.01/Images/02.201904011170.01.0002.11083200082.jpg
./AI基线资料/灰色/正常组/胡家琴_02.201905090958.01/Images/02.201905090958.01.0005.10223600454.jpg
./AI基线资料/灰色/正常组/高芹_02.201906160407.01/Images/02.201906160407.01.0008.10165600180.jpg
./AI基线资料/灰色/正常组/李采叶_02.202109300947.01/Images/02.202109300947.01.21296.0003.12052400719.jpg
./AI基线资料/灰色/正常组/姜萌_02.202109301084.01/Images/02.202109301084.01.21288.0005.11233800029.jpg
./AI基线资料/灰色/正常组/朱玉茹_02.201906190529.

./AI基线资料/灰色/正常组/马丽华_02.201905140508.01/Images/02.201905140508.01.0004.09405600211.jpg
./AI基线资料/灰色/正常组/刘爱军_02.202109020529.01/Images/02.202109020529.01.21296.0004.09303400755.jpg
./AI基线资料/灰色/正常组/马军红_02.202109130865.01/Images/02.202109130865.01.21296.0005.09531800475.jpg
./AI基线资料/灰色/正常组/孙盼_02.201904220507.01/Images/02.201904220507.01.0003.09092200844.jpg
./AI基线资料/灰色/正常组/温馨_02.202109080878.01/Images/02.202109080878.01.21288.0003.09502600284.jpg
./AI基线资料/灰色/正常组/肖遥_02.201906191084.01/Images/02.201906191084.01.0003.10584600854.jpg
./AI基线资料/灰色/正常组/赵娣_02.201906281442.01/Images/02.201906281442.01.0005.14143700366.jpg
./AI基线资料/灰色/正常组/鲍姝蒙_02.202109171111.01/Images/02.202109171111.01.21288.0005.11274700422.jpg
./AI基线资料/灰色/正常组/徐兴梅_02.201905060914.01/Images/02.201905060914.01.0008.10023100793.jpg
./AI基线资料/灰色/正常组/王媛媛_02.201905241477.01/Images/02.201905241477.01.0005.14273400510.jpg
./AI基线资料/灰色/正常组/韩志敏_02.202109091172.02/Images/02.202109091172.02.21288.0009.11541100260.jpg
./AI基线资料/灰色/正常组/刘东俊_02.20190

./AI基线资料/灰色/正常组/何长茵_02.201905061099.01/Images/02.201905061099.01.0004.11333600956.jpg
./AI基线资料/灰色/正常组/朱宁_02.201906170930.01/Images/02.201906170930.01.0005.10293500733.jpg
./AI基线资料/灰色/正常组/刘晓曼_02.202109140664.01/Images/02.202109140664.01.21288.0003.09413300732.jpg
./AI基线资料/灰色/正常组/张金娟_02.201905270494.01/Images/02.201905270494.01.0002.08413500150.jpg
./AI基线资料/灰色/正常组/吕静_02.201904250433.01/Images/02.201904250433.01.0008.08423800215.jpg
./AI基线资料/灰色/正常组/庞连学_02.201906281475.01/Images/02.201906281475.01.0005.14252200646.jpg
./AI基线资料/灰色/正常组/蔡淑芬_02.202109151209.01/Images/02.202109151209.01.21288.0005.11560100829.jpg
./AI基线资料/灰色/正常组/殷兰花_02.202109050162.01/Images/02.202109050162.01.21288.0003.08380600728.jpg
./AI基线资料/灰色/正常组/王雪丽_02.201906190522.01/Images/02.201906190522.01.0003.09092900979.jpg
./AI基线资料/灰色/正常组/吕彩霞_02.201906130851.01/Images/02.201906130851.01.0008.09475500814.jpg
./AI基线资料/灰色/正常组/孙钰_02.201906280895.01/Images/02.201906280895.01.0008.11145200281.jpg
./AI基线资料/灰色/正常组/李雪_02.201905051275.01/I

./AI基线资料/灰色/正常组/李景超_02.201903010269.01/Images/02.201903010269.01.0011.08354200446.jpg
./AI基线资料/灰色/正常组/谭琦_02.202109171323.01/Images/02.202109171323.01.21288.0003.14234000179.jpg
./AI基线资料/灰色/正常组/张芳芳_02.201904260818.01/Images/02.201904260818.01.0004.10161400155.jpg
./AI基线资料/灰色/正常组/王立芳_02.201905231503.01/Images/02.201905231503.01.0003.14480500520.jpg
./AI基线资料/灰色/正常组/黄秀萍_02.201905150429.01/Images/02.201905150429.01.0004.09471900056.jpg
./AI基线资料/灰色/正常组/王湛_02.201906280984.01/Images/02.201906280984.01.0003.11121700029.jpg
./AI基线资料/灰色/正常组/马红波_02.202109080855.01/Images/02.202109080855.01.21288.0008.09384800022.jpg
./AI基线资料/灰色/正常组/徐映_02.201904240836.01/Images/02.201904240836.01.0006.09584900931.jpg
./AI基线资料/灰色/正常组/王静_02.201905211011.01/Images/02.201905211011.01.0004.10170000667.jpg
./AI基线资料/灰色/正常组/王锐_02.201905231673.01/Images/02.201905231673.01.0006.15153300503.jpg
./AI基线资料/灰色/正常组/王莎莎_02.201906191143.01/Images/02.201906191143.01.0004.10555900899.jpg
./AI基线资料/灰色/正常组/王平_02.201905231276.02/Images/02

./AI基线资料/灰色/正常组/贾雪省_02.202109130336.01/Images/02.202109130336.01.21288.0005.08362300465.jpg
./AI基线资料/灰色/正常组/李文超_02.202110090709.01/Images/02.202110090709.01.21296.0007.09374000323.jpg
./AI基线资料/灰色/正常组/魏金梅_02.202110120485.01/Images/02.202110120485.01.20196.0009.08565200625.jpg
./AI基线资料/灰色/正常组/尉雪雅_02.201906280980.01/Images/02.201906280980.01.0005.11174900790.jpg
./AI基线资料/灰色/正常组/梁红_02.201904181305.01/Images/02.201904181305.01.0006.11262000193.jpg
./AI基线资料/灰色/正常组/张盼_02.201905171554.01/Images/02.201905171554.01.0004.14293100097.jpg
./AI基线资料/灰色/正常组/安红杰_02.201905231105.01/Images/02.201905231105.01.0005.10432600260.jpg
./AI基线资料/灰色/正常组/王妍_02.202109090397.01/Images/02.202109090397.01.21296.0011.10032800993.jpg
./AI基线资料/灰色/正常组/刘芳_02.201905231537.01/Images/02.201905231537.01.0003.14292500401.jpg
./AI基线资料/灰色/正常组/尹晓云_02.202109281643.01/Images/02.202109281643.01.21262.0003.17085300497.jpg
./AI基线资料/灰色/正常组/陈瑶_02.201904120432.01/Images/02.201904120432.01.0006.08431300176.jpg
./AI基线资料/灰色/正常组/金哲_02.2019051

./AI基线资料/灰色/全切组/张江丽_02.201905301860.01/Images/02.201905301860.01.0006.15353200703.jpg
./AI基线资料/灰色/全切组/白金香_02.201905051266.01/Images/02.201905051266.01.0005.14275200158.jpg
./AI基线资料/灰色/全切组/刘香梅_02.202110260966.01/Images/02.202110260966.01.20196.0003.10304300357.jpg
./AI基线资料/灰色/全切组/李梦军_02.202110210823.01/Images/02.202110210823.01.21296.0002.10070400376.jpg
./AI基线资料/灰色/全切组/宋安宁_02.201905301209.01/Images/02.201905301209.01.0001.11004300228.jpg
./AI基线资料/灰色/全切组/杨金芬_02.202109161319.01/Images/02.202109161319.01.21288.0002.17300600391.jpg
./AI基线资料/灰色/全切组/李佩华_02.202109071283.01/Images/02.202109071283.01.21288.0003.12000400022.jpg
./AI基线资料/灰色/全切组/张丽丽_02.201906141394.01/Images/02.201906141394.01.0002.14110500429.jpg
./AI基线资料/灰色/全切组/白翠荣_02.202109171647.01/Images/02.202109171647.01.21288.0006.16065900364.jpg
./AI基线资料/灰色/全切组/张爱红_02.201904231894.01/Images/02.201904231894.01.0003.16423500950.jpg
./AI基线资料/灰色/全切组/刘鸿敏_02.202109161785.01/Images/02.202109161785.01.21288.0001.17465000688.jpg
./AI基线资料/灰色/全切组/张桂

In [4]:
imgs = imgs + c_imgs
masks = masks + c_masks
type_labels = type_labels + c_type_labels
pathology_labels = pathology_labels + c_pathology_labels
name = name + c_name

In [5]:
test = ['2020','2021',"2019"]
tr_imgs,tr_masks,tr_type_labels,tr_pathology_labels,tr_name = [],[],[],[],[]
te_imgs,te_masks,te_type_labels,te_pathology_labels,te_name = [],[],[],[],[]
for i in range(len(name)):
    istest = False
    for t in test:
        if f'.{t}' in name[i]:
            istest = True
    if istest:
        te_imgs.append(imgs[i])
        te_masks.append(masks[i])
        te_type_labels.append(type_labels[i])
        te_pathology_labels.append(pathology_labels[i])
        te_name.append(name[i])
        
    else:
        tr_imgs.append(imgs[i])
        tr_masks.append(masks[i])
        tr_type_labels.append(type_labels[i])
        tr_pathology_labels.append(pathology_labels[i])
        tr_name.append(name[i])

In [6]:
## combine
def extract_name(n):
    n = n.split('_')
    n1,n2 = n[0],n[1]
    n1 += '_'
    for n22 in n2:
        if not (n22.isdigit() or n22 in ['.','C'] ):
            n1 += n22
    return n1
train_case,test_case = {},{}

for i,n in enumerate(tr_name):
    n1 = extract_name(n)
    if not train_case.__contains__(n1):
        train_case[n1] = tr_type_labels[i]
for i,n in enumerate(te_name):
    n1 = extract_name(n)
    if not test_case.__contains__(n1):
        test_case[n1] = te_type_labels[i]
def statistic(case):
    sta = {}
    for key in case:
        gray = case[key]
        if not sta.__contains__(gray):
            sta[gray] = 1
        else:
            sta[gray] += 1
    print(sta)
statistic(train_case)
statistic(test_case)

{'malign': 164, 'benign': 360, 'borderline': 46}
{'malign': 49, 'benign': 131, 'borderline': 14}


In [13]:
class mydataset(Dataset):
    def __init__(self,imgs,masks,types_labels,pathology_labels,name,croptumour=False,mode='train'):
        self.imgs = imgs
        self.masks = masks
        self.tlabels = types_labels
        self.plabels = pathology_labels
        self.type_dict = {'benign':0,'borderline':1,'malign':2}
        
        if croptumour:
            self.trans = albu.Compose([
                albu.OneOf([
                    albu.Resize(height=240,width=240),
                    albu.Compose([
                        albu.Resize(height=280,width=280),
                        albu.RandomCrop(height=240,width=240)
                    ])
                ],p=1),
                albu.ShiftScaleRotate(p=0.5),
                albu.HorizontalFlip(p=0.2),
                albu.VerticalFlip(p=0.2),
            ])
            self.valtrans = albu.Compose([
                albu.Resize(height=240,width=240),
            ])
        else:
            self.trans = albu.Compose([
                albu.OneOf([
                    albu.Resize(height=320,width=480),
                    albu.Compose([
                        albu.Resize(height=360,width=520),
                        albu.RandomCrop(height=320,width=480)
                    ])
                ],p=1),
                albu.ShiftScaleRotate(p=0.5),
                albu.HorizontalFlip(p=0.2),
                albu.VerticalFlip(p=0.2),
            ])
            self.valtrans = albu.Compose([
                albu.Resize(height=320,width=480),
            ])
        self.croptumour = croptumour
        self.mode = mode
        self.name = name
        
    def __getitem__(self,index):
        
        img = self.imgs[index]
        mask = self.masks[index]
        mask[mask!=0] = 1
        
        if not self.croptumour:
            if self.mode=='train':
                trans = self.trans(image=img,mask=mask)
                img,mask = trans['image'],trans['mask']
            else:
                trans = self.valtrans(image=img,mask=mask)
                img,mask = trans['image'],trans['mask']

            tlabel = self.type_dict[self.tlabels[index]]
            
            if len(img.shape) == 2:
                img = torch.tensor(img/255).float().unsqueeze(0)
            elif len(img.shape) == 3:
                img = torch.tensor(img/255).float().permute(2,0,1)
            mask = torch.tensor(mask).long()
            tlabel = torch.tensor(tlabel).long()
            return img,mask,tlabel,self.name[index]
        else:
            img = self.cutedge(img,mask)
            if self.mode == 'train':
                img = self.trans(image=img)['image']
            else:
                img = self.valtrans(image=img)['image']
            tlabel = self.type_dict[self.tlabels[index]]
            if len(img.shape) == 2:
                img = torch.tensor(img/255).float().unsqueeze(0)
            elif len(img.shape) == 3:
                img = torch.tensor(img/255).float().permute(2,0,1)
            tlabel = torch.tensor(tlabel).long()
            return img,tlabel,self.name[index]
    def __len__(self):
        return len(self.imgs)
    
    def cutedge(self,img,mask):
        tmp1 = np.max(mask,axis=0)
        tmp2 = np.max(mask,axis=1)
        for a,t in enumerate(tmp1):
            if t != 0:
                break
        for b,t in enumerate(tmp1[::-1]):
            if t != 0:
                break
        for c,t in enumerate(tmp2):
            if t != 0:
                break
        for d,t in enumerate(tmp2[::-1]):
            if t != 0:
                break
        img = img[c:len(tmp2)-d,a:len(tmp1)-b]
        return img

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def one_hot(label, n_classes, requires_grad=True):
    """Return One Hot Label"""
    divce = label.device
    one_hot_label = torch.eye(
        n_classes, device=device, requires_grad=requires_grad)[label]
    one_hot_label = one_hot_label.transpose(1, 3).transpose(2, 3)

    return one_hot_label


class BoundaryLoss(nn.Module):
    """Boundary Loss proposed in:
    Alexey Bokhovkin et al., Boundary Loss for Remote Sensing Imagery Semantic Segmentation
    https://arxiv.org/abs/1905.07852
    """

    def __init__(self, theta0=3, theta=5):
        super().__init__()

        self.theta0 = theta0
        self.theta = theta

    def forward(self, pred, gt):
        """
        Input:
            - pred: the output from model (before softmax)
                    shape (N, C, H, W)
            - gt: ground truth map
                    shape (N, H, w)
        Return:
            - boundary loss, averaged over mini-bathc
        """

        n, c, _, _ = pred.shape

        # softmax so that predicted map can be distributed in [0, 1]
        pred = torch.softmax(pred, dim=1)

        # one-hot vector of ground truth
        one_hot_gt = one_hot(gt, c)
        # boundary map
        gt_b = F.max_pool2d(
            1 - one_hot_gt, kernel_size=self.theta0, stride=1, padding=(self.theta0 - 1) // 2)
        gt_b -= 1 - one_hot_gt

        pred_b = F.max_pool2d(
            1 - pred, kernel_size=self.theta0, stride=1, padding=(self.theta0 - 1) // 2)
        pred_b -= 1 - pred

        # extended boundary map
        gt_b_ext = F.max_pool2d(
            gt_b, kernel_size=self.theta, stride=1, padding=(self.theta - 1) // 2)

        pred_b_ext = F.max_pool2d(
            pred_b, kernel_size=self.theta, stride=1, padding=(self.theta - 1) // 2)

        # reshape
        gt_b = gt_b.view(n, c, -1)
        pred_b = pred_b.view(n, c, -1)
        gt_b_ext = gt_b_ext.view(n, c, -1)
        pred_b_ext = pred_b_ext.view(n, c, -1)

        # Precision, Recall
        P = torch.sum(pred_b * gt_b_ext, dim=2) / (torch.sum(pred_b, dim=2) + 1e-7)
        R = torch.sum(pred_b_ext * gt_b, dim=2) / (torch.sum(gt_b, dim=2) + 1e-7)

        # Boundary F1 Score
        BF1 = 2 * P * R / (P + R + 1e-7)

        # summing BF1 Score for each class and average over mini-batch
        loss = torch.mean(1 - BF1)

        return loss

### 1 detction tumour

In [19]:
tr_dataset = mydataset(tr_imgs,tr_masks,tr_type_labels,tr_pathology_labels,tr_name)
tr_dataloader = DataLoader(dataset=tr_dataset,batch_size=4,shuffle=True,drop_last=True)
te_dataset = mydataset(te_imgs,te_masks,te_type_labels,te_pathology_labels,te_name,mode='test')
te_dataloader = DataLoader(dataset=te_dataset,batch_size=1,shuffle=True)

In [20]:
device='cuda:0'
# dmodel = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True)
# dmodel.backbone.conv1 = nn.Conv2d(1,64,7,2,3,bias=False)
# dmodel.classifier[4] = nn.Conv2d(256,2,1,1,bias=False)

dmodel = segmentor.ResNetUNet(n_class=2,pretrained=True)
dmodel = torch.load('./segmentator_boundaryloss_graycolor.pth',map_location=device)
dmodel = dmodel.to(device)
cri = nn.CrossEntropyLoss()
bl = BoundaryLoss()
opt = torch.optim.AdamW(dmodel.parameters(),lr=1e-5,weight_decay=3e-5)

In [28]:
def train(dl):
    epoch_loss = 0
    dmodel.train()
    for i,(x,m,t,n) in enumerate(dl):
        x,m = x.to(device),m.to(device)
        p = dmodel(x)
#         bloss = bl(p,m)
        
        loss = cri(p,m) + bl(p,m)
#         loss = cri(p,m)
        epoch_loss += loss.item() / len(dl)
        
        opt.zero_grad();loss.backward();opt.step()
        
        print('\r{}/{},train_loss:{:.4f}'.format(i,len(dl),loss.item()),end='',flush=True)
    return epoch_loss
def test(dl,plot=False):
    epoch_loss = 0
    
    preds,ys=[],[]
    ious = []
    dmodel.eval()
    for i,(x,m,t,n) in enumerate(dl):
        x,m = x.to(device),m.to(device)
        
        
        with torch.no_grad():
            p = dmodel(x)
            loss = cri(p,m)
            epoch_loss += loss.item() / len(dl)
            
            ious.append(iou(p,m))
        if plot:
            plt.figure(figsize=(10,3))
            plt.subplot(1,3,1);plt.axis("off")
            plt.imshow(x.detach().cpu().numpy()[0,0],cmap='gray')
            
            plt.subplot(1,3,2);plt.axis("off")
            p_ = torch.argmax(p,dim=1)
            plt.imshow(p_.detach().cpu().numpy()[0],cmap='gray')
#             plt.subplot(1,3,3);
#             plt.imshow(m.detach().cpu().numpy()[0],cmap='gray')
#             plt.show()
            tmp = x.detach().cpu().numpy()[0]
            tmp = tmp.transpose(1,2,0)
            tmp = np.uint8(tmp * 255)
            mask = m[0].detach().cpu().numpy().reshape(320,480,1)
            mask = np.uint8(mask * 255)
            mask = np.concatenate([np.zeros_like(mask),np.zeros_like(mask),mask],axis=2)
            merge = cv2.addWeighted(tmp,1,mask,0.5,1)
            mask = p_[0].detach().cpu().numpy().reshape(320,480,1)
            mask = np.uint8(mask * 255)
            contours = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)[-2]
            merge = cv2.drawContours(merge,contours,-1,thickness=2,color=(0,255,255))
#             cv2.imwrite('./forpaper/2/{}.png'.format(np.random.randint(100)),merge)
            plt.subplot(1,3,3);
            plt.imshow(merge[:,:,::-1],cmap='gray')
            plt.axis("off")
            plt.savefig(f"./output/0606/{n[0]}.png",dpi=100)
            plt.show()
            print(tmp.shape)
        print('\r{}/{},test_loss:{:.4f}'.format(i,len(dl),loss.item()),end='',flush=True)
    ious = np.mean(np.concatenate(ious))
    return epoch_loss,{'iou':ious}
def iou(p,gt):
    p_ = torch.argmax(p,dim=1)
    
    a = torch.sum(p_ * gt,dim=[1,2])
    b = torch.sum(p_,dim=[1,2]) + torch.sum(gt,dim=[1,2]) - a
    iou = (a + 1) / (b + 1)
    return iou.detach().cpu().numpy()

In [24]:
# test_loss,metric = test(te_dataloader,plot=True=


In [16]:
test_loss,metric = test(te_dataloader)
best_iou = metric['iou']
print('best_iou',best_iou)
for e in range(100):
    train_loss = train(tr_dataloader)
    test_loss,metric = test(te_dataloader)
    print('-----------------')
    print('train_loss:{:.4f},test_loss:{:.4f}'.format(train_loss,test_loss))
    print('iou:{:.4f}'.format(metric['iou']))    
    if best_iou < metric['iou']:
        best_iou = metric['iou']
        torch.save(dmodel,'./segmentator_boundaryloss_graycolor.pth')
        print('==> saved model') 

166/167,test_loss:0.0471best_iou 0.85067093
166/167,test_loss:0.04120-----------------
train_loss:0.3870,test_loss:0.0348
iou:0.8443
166/167,test_loss:0.01695-----------------
train_loss:0.3691,test_loss:0.0348
iou:0.8457
166/167,test_loss:0.03225-----------------
train_loss:0.3663,test_loss:0.0343
iou:0.8461
166/167,test_loss:0.04797-----------------
train_loss:0.3605,test_loss:0.0339
iou:0.8455
166/167,test_loss:0.06115-----------------
train_loss:0.3517,test_loss:0.0356
iou:0.8417
166/167,test_loss:0.02512-----------------
train_loss:0.3468,test_loss:0.0340
iou:0.8476
166/167,test_loss:0.01788-----------------
train_loss:0.3407,test_loss:0.0341
iou:0.8461
166/167,test_loss:0.01700-----------------
train_loss:0.3376,test_loss:0.0350
iou:0.8498
166/167,test_loss:0.01414-----------------
train_loss:0.3348,test_loss:0.0325
iou:0.8525
==> saved model
166/167,test_loss:0.03841-----------------
train_loss:0.3332,test_loss:0.0324
iou:0.8505
166/167,test_loss:0.03233-----------------
train_l

KeyboardInterrupt: 

In [29]:
test_loss,metric = test(te_dataloader,plot=True)
